# PANDAS

## Like R in Python but better

Today we'll cover
- What are DataFrames  
  - Indeces vs. Columns  
  - Setting the index  
  - Reference by name/Reference by position  
- Basic operations on DataFrames  
  - Filter  
  - Iterate  
  - Transform to other datatypes  
- Advanced operations  
  - join, merge, append  
  - Multi index  

# Prologue

## DataTypes 

### Basic Types
- Int
  - 1
  - 42
- Float
  - 2.2222
  - 3.14359
- Char
  - '2'
  - 'a'
  - '"'


### Structured Types
- List
  - `(a -> (b -> (c -> ...)))`
- Array  
  - `[a, b, c]`
- Hashtable
  - `[(1 -> 'a'), (2 -> 'b'), (3 -> 'c')]`

### Abstractions
- Char + List = String
- Array + List = ArrayList (Creative Right?)

# Python is a liar
- Strings are secretly Lists
- Lists are secretly hashtables
- Dicts are openly hashtables
- Everything is hashtables

# What is a Dataframe?

A bunch of hashtables 

# What is special about them?

In [70]:
import numpy as np
import pandas as pd
from pprint import pprint
# Let's say we want to collect information about particulars days
# and access that information based on the date

# We want to know the day of the week it was
day_of_week = {
    pd.Timestamp('20200101'): 'Wednesday',
    pd.Timestamp('20200102'): 'Thursday',
    pd.Timestamp('20200103'): 'Friday',
    pd.Timestamp('20200104'): 'Saturday',
    pd.Timestamp('20200105'): 'Sunday'
}

day_of_week

{Timestamp('2020-01-01 00:00:00'): 'Wednesday',
 Timestamp('2020-01-02 00:00:00'): 'Thursday',
 Timestamp('2020-01-03 00:00:00'): 'Friday',
 Timestamp('2020-01-04 00:00:00'): 'Saturday',
 Timestamp('2020-01-05 00:00:00'): 'Sunday'}

In [71]:
# We also want to know the high tempurature
high_temp = {
    pd.Timestamp('20200101'): 48,
    pd.Timestamp('20200102'): 54,
    pd.Timestamp('20200103'): 45,
    pd.Timestamp('20200104'): 61,
    pd.Timestamp('20200105'): 55
}
high_temp

{Timestamp('2020-01-01 00:00:00'): 48,
 Timestamp('2020-01-02 00:00:00'): 54,
 Timestamp('2020-01-03 00:00:00'): 45,
 Timestamp('2020-01-04 00:00:00'): 61,
 Timestamp('2020-01-05 00:00:00'): 55}

In [72]:
# And the low tempurature 
low_temp = {
    pd.Timestamp('20200101'): 30,
    pd.Timestamp('20200102'): 38,
    pd.Timestamp('20200103'): 33,
    pd.Timestamp('20200104'): 45,
    pd.Timestamp('20200105'): 30
}
low_temp

{Timestamp('2020-01-01 00:00:00'): 30,
 Timestamp('2020-01-02 00:00:00'): 38,
 Timestamp('2020-01-03 00:00:00'): 33,
 Timestamp('2020-01-04 00:00:00'): 45,
 Timestamp('2020-01-05 00:00:00'): 30}

In [73]:
# This quickly gets difficult to work with
# Let's say you want to compare weekday and weekend high temps
# You'd have to do something terrible like the following
weekends = []
weekdays = []

# Collect the info from one dict
for timestamp, day in day_of_week.items():
    if day == 'Saturday' or day == 'Sunday':
        weekends.append(timestamp)
    else:
        weekdays.append(timestamp)

weekend_temps = []
weekday_temps = []
# Use it to collect info from another dict
for timestamp in weekends:
    weekend_temps.append(high_temp[timestamp])
for timestamp in weekdays:
    weekday_temps.append(high_temp[timestamp])

print("Weekend average {}".format(np.mean(weekend_temps)))
print("Weekday average {}".format(np.mean(weekday_temps)))

Weekend average 58.0
Weekday average 49.0


In [74]:
## You want to access the data all in the same place
df = pd.DataFrame({'day_of_week': day_of_week,
                   'high_temp': high_temp,
                   'low_temp': low_temp})
df

,day_of_week,high_temp,low_temp
2020-01-01,Wednesday,48,30
2020-01-02,Thursday,54,38
2020-01-03,Friday,45,33
2020-01-04,Saturday,61,45
2020-01-05,Sunday,55,30


In [75]:
# How do you access it?
# Indexes (Rows) and Columns (Columns)
print(df.index)
print(df.columns)

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05'],
              dtype='datetime64[ns]', freq=None)
Index(['day_of_week', 'high_temp', 'low_temp'], dtype='object')


In [76]:
# So how would we find the weekday/weekend temps?
weekend_days = ['Saturday', 'Sunday']
weekend_data, weekday_data = df.groupby(df.day_of_week.isin(weekend_days))

print("Weekend average {}".format(weekend_data[1].high_temp.mean()))
print("Weekday average {}".format(weekday_data[1].high_temp.mean()))

Weekend average 49.0
Weekday average 58.0


In [77]:
# Step 1: Accessing the day_of_week information
day_of_week = df.day_of_week
day_of_week

2020-01-01    Wednesday
2020-01-02     Thursday
2020-01-03       Friday
2020-01-04     Saturday
2020-01-05       Sunday
Name: day_of_week, dtype: object

In [78]:
# Step 2: Determining if it's a weekend
is_weekend = day_of_week.isin(weekend_days)
is_weekend

2020-01-01    False
2020-01-02    False
2020-01-03    False
2020-01-04     True
2020-01-05     True
Name: day_of_week, dtype: bool

In [79]:
# Step 3: Group the data based on those True/False values
group1, group2 = df.groupby(is_weekend)
# groupby returns a tuple e.g. (group_value, dataframe)
# In this case group_value is True or False but it could be any value
print(group1[0])
print("-----------")
print(group1[1])


False
-----------
           day_of_week  high_temp  low_temp
2020-01-01   Wednesday         48        30
2020-01-02    Thursday         54        38
2020-01-03      Friday         45        33


In [80]:
# Step 4: Get the high_temp from the group
high_temps = group1[1].high_temp
high_temps

2020-01-01    48
2020-01-02    54
2020-01-03    45
Name: high_temp, dtype: int64

In [81]:
# Step 5: Calculate the average
high_temps.mean()

49.0

# Multi Indexes
- The best way to handle hierarchical categories 

In [82]:
# What is a MutliIndex
weekday_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
tups = [('weekend', x) for x in weekend_days] + [('weekday', x) for x in weekday_days] 
idx = pd.MultiIndex.from_tuples(tups)
idx

MultiIndex([('weekend',  'Saturday'),
            ('weekend',    'Sunday'),
            ('weekday',    'Monday'),
            ('weekday',   'Tuesday'),
            ('weekday', 'Wednesday'),
            ('weekday',  'Thursday'),
            ('weekday',    'Friday')],
           )

In [83]:
# How to add it to a dataframe
# Requires a column for each level of the index
new_col = pd.Series({x[1]: x[0] for x in tups}, name="type_of_day")
# Add a column. We'll cover what's happening here later
mdf = df.set_index('day_of_week').T.append(new_col).T.reset_index().rename({'index':  'day_of_week'}, axis=1)
# Now set the new index
mdf.set_index(['type_of_day', 'day_of_week'])

high_temp low_temp
type_of_day day_of_week                   
weekday     Wednesday          48       30
            Thursday           54       38
            Friday             45       33
weekend     Saturday           61       45
            Sunday             55       30
weekday     Monday            NaN      NaN
            Tuesday           NaN      NaN

In [84]:
# So NOW how would we find the weekday/weekend temps?
print("Weekday average {}".format(mdf.query('type_of_day == "weekday"').high_temp.mean()))
print("Weekend average {}".format(mdf.query('type_of_day == "weekend"').high_temp.mean()))

Weekday average 49.0
Weekend average 58.0


# Adding to a DataFrame
Finding the average tempurature was very exciting but it's left a few things unanswered

Who are the days for?

In [85]:
for_whom = {
    'Sunday': 'The girls',
    'Monday': 'The birds',
    'Tuesday': 'The non-binary',
    'Wednesday': 'The camels',
    'Thursday': 'The dogs',
    'Friday': 'The cats',
    'Saturday': 'The boys',
}
for_whom

{'Sunday': 'The girls',
 'Monday': 'The birds',
 'Tuesday': 'The non-binary',
 'Wednesday': 'The camels',
 'Thursday': 'The dogs',
 'Friday': 'The cats',
 'Saturday': 'The boys'}

In [86]:
for_whom = pd.Series(for_whom, name='for_whom')
for_whom

Sunday            The girls
Monday            The birds
Tuesday      The non-binary
Wednesday        The camels
Thursday           The dogs
Friday             The cats
Saturday           The boys
Name: for_whom, dtype: object

In [87]:
df.append(for_whom)

,day_of_week,high_temp,low_temp,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
2020-01-01 00:00:00,Wednesday,48.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02 00:00:00,Thursday,54.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03 00:00:00,Friday,45.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04 00:00:00,Saturday,61.0,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05 00:00:00,Sunday,55.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
for_whom,NaN,NaN,NaN,The cats,The birds,The boys,The girls,The dogs,The non-binary,The camels


Thats not right

In [88]:
# We need to transform the data frame so pandas can match the new data to the original
idf = df.set_index('day_of_week')
idf

,high_temp,low_temp
day_of_week,,
Wednesday,48,30
Thursday,54,38
Friday,45,33
Saturday,61,45
Sunday,55,30


In [89]:
idf.append(for_whom)

,high_temp,low_temp,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
day_of_week,,,,,,,,,
Wednesday,48.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Thursday,54.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Friday,45.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Saturday,61.0,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sunday,55.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
for_whom,NaN,NaN,The cats,The birds,The boys,The girls,The dogs,The non-binary,The camels


Still not right

In [90]:
idft = idf.T
idft

day_of_week,Wednesday,Thursday,Friday,Saturday,Sunday
high_temp,48,54,45,61,55
low_temp,30,38,33,45,30


In [91]:
idft = idft.append(for_whom)
idft

,Wednesday,Thursday,Friday,Saturday,Sunday,Monday,Tuesday
high_temp,48,54,45,61,55,NaN,NaN
low_temp,30,38,33,45,30,NaN,NaN
for_whom,The camels,The dogs,The cats,The boys,The girls,The birds,The non-binary


That's better

In [92]:
# To get it back to the previous format we undo to the transformations
idf = idft.T.reset_index()
idf

,index,high_temp,low_temp,for_whom
0,Wednesday,48,30,The camels
1,Thursday,54,38,The dogs
2,Friday,45,33,The cats
3,Saturday,61,45,The boys
4,Sunday,55,30,The girls
5,Monday,NaN,NaN,The birds
6,Tuesday,NaN,NaN,The non-binary


# Notice
- The Week days now have the column label 'index'
- The original label was lost when we set them to be the index.
- We lost the original index values. 
- We didn't have temps for Monday or Tuesday so those have been filled with NaNs

In [93]:
# To retain the original index we need to save it in a column before setting the new index
df['date'] = df.index
df

,day_of_week,high_temp,low_temp,date
2020-01-01,Wednesday,48,30,2020-01-01
2020-01-02,Thursday,54,38,2020-01-02
2020-01-03,Friday,45,33,2020-01-03
2020-01-04,Saturday,61,45,2020-01-04
2020-01-05,Sunday,55,30,2020-01-05


In [94]:
# Then proceed as before
sdf = df.set_index('day_of_week').T.append(for_whom).T.reset_index()
sdf

,index,high_temp,low_temp,date,for_whom
0,Wednesday,48,30,2020-01-01,The camels
1,Thursday,54,38,2020-01-02,The dogs
2,Friday,45,33,2020-01-03,The cats
3,Saturday,61,45,2020-01-04,The boys
4,Sunday,55,30,2020-01-05,The girls
5,Monday,NaN,NaN,NaT,The birds
6,Tuesday,NaN,NaN,NaT,The non-binary


# Joining a Dataframe to a Dataframe

In [95]:
# In the last section we joined a 1-dimensional series to our dataframe, but what if we have a second dataframe
df2 = pd.DataFrame({
   'day_of_week': ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'],
   'date': [
       pd.Timestamp('20200621'),
       pd.Timestamp('20200622'),
       pd.Timestamp('20200623'),
       pd.Timestamp('20200624'),
       pd.Timestamp('20200625'),
       pd.Timestamp('20200626'),
       pd.Timestamp('20200627'),
   ],
    'high_temp': [81, 81, 81, 82, 82, 82, 83],
    'low_temp': [66, 66, 66, 66, 67, 67, 68]
})
df2

,day_of_week,date,high_temp,low_temp
0,Sunday,2020-06-21,81,66
1,Monday,2020-06-22,81,66
2,Tuesday,2020-06-23,81,66
3,Wednesday,2020-06-24,82,66
4,Thursday,2020-06-25,82,67
5,Friday,2020-06-26,82,67
6,Saturday,2020-06-27,83,68


In [96]:
df3 = pd.DataFrame({
   'day_of_week': ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'],
   'date': [
       pd.Timestamp('20201101'),
       pd.Timestamp('20201102'),
       pd.Timestamp('20201103'),
       pd.Timestamp('20201104'),
       pd.Timestamp('20201105'),
       pd.Timestamp('20201106'),
       pd.Timestamp('20201107'),
   ],
    'high_temp': [59, 58, 58, 57, 57, 57, 56],
    'low_temp': [46, 45, 45, 45, 45, 44, 44]
})
df3

,day_of_week,date,high_temp,low_temp
0,Sunday,2020-11-01,59,46
1,Monday,2020-11-02,58,45
2,Tuesday,2020-11-03,58,45
3,Wednesday,2020-11-04,57,45
4,Thursday,2020-11-05,57,45
5,Friday,2020-11-06,57,44
6,Saturday,2020-11-07,56,44


# How do we combine these datasets?

In [97]:
# The simplest is concat
pd.concat([sdf, df2, df3])

,index,high_temp,low_temp,date,for_whom,day_of_week
0,Wednesday,48,30,2020-01-01,The camels,NaN
1,Thursday,54,38,2020-01-02,The dogs,NaN
2,Friday,45,33,2020-01-03,The cats,NaN
3,Saturday,61,45,2020-01-04,The boys,NaN
4,Sunday,55,30,2020-01-05,The girls,NaN
5,Monday,NaN,NaN,NaT,The birds,NaN
6,Tuesday,NaN,NaN,NaT,The non-binary,NaN
0,NaN,81,66,2020-06-21,NaN,Sunday
1,NaN,81,66,2020-06-22,NaN,Monday
2,NaN,81,66,2020-06-23,NaN,Tuesday


- Stick one dataframe onto the end of another.
- Notice that the data isn't perfectly aligned. 'day_of_week' and 'index' should be the same column

In [98]:
# To fix this rename 'index'
sdf = sdf.rename({'index': 'day_of_week'}, axis=1)
# Axis 1 indicates to operate on 'columns'. The default is 'index'
sdf

,day_of_week,high_temp,low_temp,date,for_whom
0,Wednesday,48,30,2020-01-01,The camels
1,Thursday,54,38,2020-01-02,The dogs
2,Friday,45,33,2020-01-03,The cats
3,Saturday,61,45,2020-01-04,The boys
4,Sunday,55,30,2020-01-05,The girls
5,Monday,NaN,NaN,NaT,The birds
6,Tuesday,NaN,NaN,NaT,The non-binary


In [99]:
pd.concat([sdf, df2, df3])

,day_of_week,high_temp,low_temp,date,for_whom
0,Wednesday,48,30,2020-01-01,The camels
1,Thursday,54,38,2020-01-02,The dogs
2,Friday,45,33,2020-01-03,The cats
3,Saturday,61,45,2020-01-04,The boys
4,Sunday,55,30,2020-01-05,The girls
5,Monday,NaN,NaN,NaT,The birds
6,Tuesday,NaN,NaN,NaT,The non-binary
0,Sunday,81,66,2020-06-21,NaN
1,Monday,81,66,2020-06-22,NaN
2,Tuesday,81,66,2020-06-23,NaN


- Better but still not perfect
- The values of 'for_whom' have a one-to-one correspondance to the values in 'day_of_week' so we can infer them

# We drop columns that aren't in all the dataframes

In [100]:
big_df = pd.concat([sdf, df2, df3], join='inner') # Default value for `join` is 'outer'
big_df

,day_of_week,high_temp,low_temp,date
0,Wednesday,48,30,2020-01-01
1,Thursday,54,38,2020-01-02
2,Friday,45,33,2020-01-03
3,Saturday,61,45,2020-01-04
4,Sunday,55,30,2020-01-05
5,Monday,NaN,NaN,NaT
6,Tuesday,NaN,NaN,NaT
0,Sunday,81,66,2020-06-21
1,Monday,81,66,2020-06-22
2,Tuesday,81,66,2020-06-23


In [101]:
# Then add the 'for_whom' information like we did before
big_df = big_df.set_index('day_of_week').T.append(for_whom).T.reset_index()
big_df

,day_of_week,high_temp,low_temp,date,for_whom
0,Wednesday,48,30,2020-01-01,The camels
1,Thursday,54,38,2020-01-02,The dogs
2,Friday,45,33,2020-01-03,The cats
3,Saturday,61,45,2020-01-04,The boys
4,Sunday,55,30,2020-01-05,The girls
5,Monday,NaN,NaN,NaT,The birds
6,Tuesday,NaN,NaN,NaT,The non-binary
7,Sunday,81,66,2020-06-21,The girls
8,Monday,81,66,2020-06-22,The birds
9,Tuesday,81,66,2020-06-23,The non-binary


# Or we could perform database style merging

In [102]:
# You can 'merge'
pd.merge(sdf, df2, on='day_of_week', how='outer')

,day_of_week,high_temp_x,low_temp_x,date_x,for_whom,date_y,high_temp_y,low_temp_y
0,Wednesday,48,30,2020-01-01,The camels,2020-06-24,82,66
1,Thursday,54,38,2020-01-02,The dogs,2020-06-25,82,67
2,Friday,45,33,2020-01-03,The cats,2020-06-26,82,67
3,Saturday,61,45,2020-01-04,The boys,2020-06-27,83,68
4,Sunday,55,30,2020-01-05,The girls,2020-06-21,81,66
5,Monday,NaN,NaN,NaT,The birds,2020-06-22,81,66
6,Tuesday,NaN,NaN,NaT,The non-binary,2020-06-23,81,66


In [103]:
# Or you could join
sdf.set_index('day_of_week').join(df2.set_index('day_of_week'), lsuffix='_left')

,high_temp_left,low_temp_left,date_left,for_whom,date,high_temp,low_temp
day_of_week,,,,,,,
Wednesday,48,30,2020-01-01,The camels,2020-06-24,82,66
Thursday,54,38,2020-01-02,The dogs,2020-06-25,82,67
Friday,45,33,2020-01-03,The cats,2020-06-26,82,67
Saturday,61,45,2020-01-04,The boys,2020-06-27,83,68
Sunday,55,30,2020-01-05,The girls,2020-06-21,81,66
Monday,NaN,NaN,NaT,The birds,2020-06-22,81,66
Tuesday,NaN,NaN,NaT,The non-binary,2020-06-23,81,66


# Notice:
- Columns other than 'day_of_week' and 'for_whom' each column has a duplicate entry
- One for the 'left' dataframe (x)
- One for the 'right' dataframe (y)
- 'merge' and 'join' are largely equivalent. Join operates on the 'index' where as merge operates on a specified column

# That's all folks